Install libraries:

In [1]:
!pip install --upgrade transformers
!pip install googletrans==3.1.0a0
!pip install sentencepiece
!pip install rouge-score
!pip install docx
!pip install rouge
!pip3 install textstat
!pip install google
!pip install bs4
!pip install requests
!pip install sumy
!pip install python-docx

     |████████████████████████████████| 1.8MB 5.6MB/s 
     |████████████████████████████████| 890kB 22.8MB/s 
     |████████████████████████████████| 2.9MB 23.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=afcc76397cae954da52f56a9d27f6e3694f4b15f2816479a35910db217b1a4c6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 51kB 589kB/s 
     |████████████████████████████████| 1.0MB 6.2MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 102kB 6.9MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp36-none-any.whl size=16368 sha256=fa63a85c5b462d50b4c428f4a1bec13123c4556957267debd41dff69e85bb830
  Stored in directory: /root/.cache/pip/wheels

Import libraries:

In [2]:
import re
import torch
import textstat
from tqdm import tqdm
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from difflib import SequenceMatcher

from rouge_score import rouge_scorer
from googletrans import Translator
from rouge import Rouge
from requests import get
from tqdm import tqdm
import docx

import nltk
import pandas
import random
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

from nltk.tokenize import sent_tokenize, word_tokenize

# web parsing
from bs4 import BeautifulSoup
from bs4.element import Tag
#/web parsing

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

CDFs:

In [6]:
def get_response(input_text, num_return_sequences, max_length):
    
    batch = tokenizer.prepare_seq2seq_batch([input_text], truncation=True, padding='longest', max_length=100, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch, max_length=max_length, num_beams=10, num_return_sequences=num_return_sequences, temperature=1)
    
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return tgt_text

def text_normalize(txt):
    
    processed_text = re.sub('[^a-zA-Z]', ' ', txt)
    processed_text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",processed_text)
    processed_text=re.sub("(\\d|\\W)+"," ",processed_text)

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(processed_text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if not word in stop_words]
    tokens = [i for i in tokens if (tags(i) in ['NN', 'NNP', 'NNS', 'NNPS'])]

    return tokens

def get_scores(report_summary, final_text):
    scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)
    scores = scorer.score(' '.join(text_normalize(report_summary)).lower(), ' '.join(text_normalize(final_text)).lower())
    scores = round(list(list(scores.values())[0])[2], 2)
    
    return scores

def tags(x):
    return nltk.pos_tag(nltk.word_tokenize(x))[0][1]

def readability_ratio(x,y):

  x = textstat.syllable_count(x)
  y = textstat.syllable_count(y)
  
  return(round(y/x,2))

def getText(filename):
    
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    
    return '\n'.join(fullText)                     

Feature toggles:

In [31]:
translator = True

Define the model:

In [5]:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

#### Unit test

Input text:

In [24]:
original = '''|Problem|

We live in the century of information explosion and multitasking. Companies and people have overindulged in information. Some are finding it more difficult than ever to decide, learn and deliver due to overload and multitasking. 
It is found that diverted attention can result in reduced performance and stress. Research has shown that interruption can cause higher completion time and double task error rate. We can’t completely avoid multitasking but…we can 
adopt our activities, including education to the new environment. That's where microlearning technology is applied. Our learning process can be separated into short skill based independent modules. It turned out that it should (!) 
be separated. Recent research has shown that we learn more and retain it better when we study in short-focused bursts than when we are forced to sit through long classes: both kids and grown-ups. 

|Solution|

Microlearning is the approach for skill-based learning and education which deals with small learning units. Microlearning involves short-term-focused educational strategies. In a wider sense, it is the way more and more people are 
actually doing informal learning and gaining knowledge in micro-content, micro-media or multitasking environments. Microlearning opens up an opportunity of flexible personalized learning as well. Some applications allow easy managing 
strategy for individual learners with full analytics available to track their progress retention and engagement. Microlearning strategy can be built on interactive microlearning modules that you can create on your own. You just need 
to add texts and images to the template and you’re ready to go. 

|Applications|

An organization can use microlearning to help new hires learn their way around the facility. It provides fast answers to many commonly asked questions and recount their company’s history from the launch date up to the current day. 
The enormous growth of smartphone ownership makes it the perfect learning and development solution for any skilled workforce whether it is retail hospitality enterprise or any other industry. Microlearning may be provided in the classic 
way through a webinar in the workplace. However mobile learning brings benefits such as push notifications for lesson reminders, ability to deliver microlessons that are always up to date direct to learners’ pockets. Additionally, quizzes 
with multiple choice options on cell phones by use of SMS or mobile applications.

|Product format|

Microlearning content types include video tutorials, audio podcasts, interactions games and RSS feeds, where learners can participate by threaded discussions or online meetings. Other activities include viewing a flashcards, memorizing definition 
or formula, playful learning with micro games, composing a haiku or a short poem. Some microlearning activities can be embedded in the working day. For example, PC screensaver may prompt the user to solve small series of simple tasks after a certain 
amount of inactivity while flashcard software helps memorizing content through spaced repetition.  

|Perspectives|

Microlearning strategy is on interactive microlearning modules that you can create on your own. Rapid change in the knowledge demand will most likely lead to the growth of microlearning impact. However, we believe that education will preserve 
some share of academicism to be a good foundation for microlearning plugins. Thank you for your attention.'''

Test paraphrase:

In [54]:
%%time
summ_list = []

for i in tqdm(original.split('.')[:-1]):
  
  z = get_response(i, 1, len(word_tokenize(i)))[0]
  z = z.replace('.', '') + '.' 
  summ_list.append(z)

  if translator == True:
    translator = Translator()
    ru_version = translator.translate(pegas_summary, dest='ru').text
    pegas_summary = translator.translate(ru_version, dest='en').text

pegas_summary = ' '.join(summ_list)

print('\n\nROUGE score:', get_scores(pegas_summary, original))
print('Complexity score:', readability_ratio(original, pegas_summary))
print('Plaugarism:', round(SequenceMatcher(None, original, pegas_summary).ratio(), 2), '\n')  

  0%|          | 0/31 [00:00<?, ?it/s]

|Problem|

We live in the century of information explosion and multitasking


  3%|▎         | 1/31 [00:00<00:19,  1.53it/s]

|Problem|

We live in the century of information explosion and multitasking
 Companies and people have overindulged in information


  6%|▋         | 2/31 [00:01<00:21,  1.38it/s]

...

 Rapid change in the knowledge demand will most likely lead to the growth of microlearning impact
 However, we believe that education will preserve 
some share of academicism to be a good foundation for microlearning plugins


 97%|█████████▋| 30/31 [00:21<00:00,  1.48it/s]

 However, we believe that education will preserve 
some share of academicism to be a good foundation for microlearning plugins
 Thank you for your attention


100%|██████████| 31/31 [00:22<00:00,  1.38it/s]

 Thank you for your attention
CPU times: user 398 ms, sys: 39.9 ms, total: 438 ms
Wall time: 22.4 s


In [21]:
pegas_summary

"|Problem|\n\nWe live in the century of information explosion and multitasking  Companies and people have overindulged in information  Some are finding it more difficult than ever to decide, learn and deliver due to overload and multitasking  \nIt is found that diverted attention can result in reduced performance and stress  Research has shown that interruption can cause higher completion time and double task error rate  We can’t completely avoid multitasking but…we can \nadopt our activities, including education to the new environment  That's where microlearning technology is applied  Our learning process can be separated into short skill based independent modules  It turned out that it should (!) \nbe separated  Recent research has shown that we learn more and retain it better when we study in short-focused bursts than when we are forced to sit through long classes: both kids and grown-ups  \n\n|Solution|\n\nMicrolearning is the approach for skill-based learning and education which d

Metrics:

In [ ]:
print('ROUGE score:', get_scores(pegas_summary, original))
print('Complexity score:', readability_ratio(original, pegas_summary))
print('Plaugarism:', round(SequenceMatcher(None, original, pegas_summary).ratio(), 2))   

ROUGE score: 0.49
Complexity score: 0.69
Plaugarism: 0.07


#### Statistical test

Parameters:

In [ ]:
sample_length = 2000

Download wiki:

In [ ]:
!wget -O wiki_corpus https://drive.google.com/u/2/uc?id=1HvzyuQhM3VGMe5yiil-0vYk0fqvZM44F&export=download

--2021-01-19 21:03:20--  https://drive.google.com/u/2/uc?id=1HvzyuQhM3VGMe5yiil-0vYk0fqvZM44F
Resolving drive.google.com (drive.google.com)... 74.125.31.113, 74.125.31.138, 74.125.31.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.31.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-c8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pi09u3fa7k67uv3enit1g3vr2e1kbbrl/1611090150000/07849071433606409146/*/1HvzyuQhM3VGMe5yiil-0vYk0fqvZM44F [following]
--2021-01-19 21:03:23--  https://doc-14-c8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pi09u3fa7k67uv3enit1g3vr2e1kbbrl/1611090150000/07849071433606409146/*/1HvzyuQhM3VGMe5yiil-0vYk0fqvZM44F
Resolving doc-14-c8-docs.googleusercontent.com (doc-14-c8-docs.googleusercontent.com)... 173.194.218.132, 2607:f8b0:400c:c14::84
Connecting to doc-14-c8-docs.googleusercontent.com (doc-14-c8-docs.googleusercontent.com)|173.1

Create dataset:

In [ ]:
df = pandas.read_excel('/content/wiki_corpus')
df = df.sample(n = 3)
df.columns = ['text']

dataset = [i[:sample_length] for i in list(df['text'])] 

Collect data:

In [ ]:
%%time

rouge_list = []
complex_list = []
plaug_list = []

for i in tqdm(dataset):

  original = i
  summ_list = []
  
  for j in original.split('.')[:-1]:
    
    try:
      z = get_response(j, 1, len(word_tokenize(j)))[0]
      z = z.replace('.', '') + '.' 
      summ_list.append(z)
    except:
      print('Error!') 
   
  pegas_summary = ' '.join(summ_list)

  if translator == True:
    translator = Translator()
    ru_version = translator.translate(pegas_summary, dest='ru').text
    pegas_summary = translator.translate(ru_version, dest='en').text

  print(pegas_summary)  

  rouge_list.append(get_scores(pegas_summary, original))
  complex_list.append(readability_ratio(original, pegas_summary))
  plaug_list.append(round(SequenceMatcher(None, original, pegas_summary).ratio(),2))    

  0%|          | 0/5 [00:00<?, ?it/s]Input length of decoder_input_ids is 1, but ``max_length`` is set to 1.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


Error!


Input length of decoder_input_ids is 1, but ``max_length`` is set to 1.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


Error!


Input length of decoder_input_ids is 1, but ``max_length`` is set to 1.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


...

Error!


 20%|██        | 1/5 [01:43<06:55, 104.00s/it]

Microelectrode array (MEAs) are devices that contain multiple (tens to thousands) microelectrodes through which neural signals are obtained or delivered, essentially serving as neural interface that connect neurons to electronic circuitry. There are two general classes of MEAs. Theory Neurons and muscle cells create ion currents through their membranes when excited, causing a change in voltage between the inside and. The MEA transduces the change in voltage from the environment carried by ion into currents carried by electrons. The media stimulates electronic currents into ionic currents. If it is a neuron or a twitch, this will cause the cell to depolarize and cause an action potential if it is a muscle cell. There are several factors that affect the size and shape of a recorded signal. The medium's electrical conductivity, capacitance, and homogeneity; the nature of contact between the cells and the MEA. The nature of the MEA electrode. Its geometry, impedance, and noise. The system'

 40%|████      | 2/5 [03:44<05:26, 108.91s/it]

The television series Mystery Science Theater 3000 has a robot named Gypsy. She is larger than the other robots. During the show's host segments and introduction, Gypsy usually only appears, but she took a seat in the theater to watch the movie "Hercules and the Captive. She only delivered a couple of "riffs" because she took the movie and realized how bad it was. The creator of the series designed and built the robot. The robot's size and ponderousness reminded him of his brother's pet turtle, so he named. The Satellite of Love's higher functions are taken care of by Gypsy. She needs to use most of her computing power to deal with this responsibility, which makes her appear slow-witted. The demand on her systems was temporarily reduced in the episode. She made a quick appearance in the theater when someone mentioned that she was a huge fan of the show's star, Richard Basehart. She became a more frequent participant in the host segments, and appeared more intelligent, even attempting t

Dig data paraphrase metrics:

In [ ]:
print('Median ROUGE:', round(np.median(rouge_list),2))
print('ROUGE dispersion:', round(np.std(rouge_list),2), '\n')

print('Median complexity:', round(np.median(complex_list),2))
print('Complexity dispersion:', round(np.std(complex_list),2), '\n')

print('Median plagiarism:', round(np.median(plaug_list),2))
print('Plagiarism plagiarism:', round(np.std(plaug_list),2), '\n')

Median ROUGE: 0.45
ROUGE dispersion: 0.05 

Median complexity: 0.59
Complexity dispersion: 0.08 

Median plagiarism: 0.11
Plagiarism plagiarism: 0.04 

